In [1]:
import tkinter as tk
import customtkinter as ctk
from tkinter import filedialog

from openpyxl import load_workbook
import pandas as pd

from PIL import Image
import numpy as np
import random

custom_font =("Calisto MT",18,'bold')
BACKGROUND_COLOR = "#B1DDC6"
global wb
clicked_list=[]
last_bt_id=None
ids_k_list = []
colm_val =[]
match = 0

known_image = ctk.CTkImage(Image.open(r"./tick.png"))
unknown_image = ctk.CTkImage(Image.open(r"./cross.png"))


class Main(ctk.CTkFrame):
    def __init__(self, master=None, **kwargs):
        ctk.CTkFrame.__init__(self, master, **kwargs)
        label1 = ctk.CTkLabel(self,text='Please choose a file',font=custom_font)
        self.label2 = ctk.CTkLabel(self, text=f"No file selected",font=custom_font, wraplength=400)
        self.exp_btn = ctk.CTkButton(self,text="Export my ExcelFile", command = self.browseFiles, width=100, height=30)
        self.button_frame = ctk.CTkFrame(self)
        self.action_frame = ctk.CTkFrame(self)

        label1.pack(padx=25, pady=10)
        self.exp_btn.pack(padx=5, pady=10)
        self.label2.pack(padx=20, pady=5)


    def browseFiles(self,event=None):
        global count, word_row, filename, df_leng, ws

        filename = filedialog.askopenfilename(initialdir = "/",
                                          title = "Select a File",
                                          filetypes = (("Text files",
                                                        "*xlsx*"),
                                                       ("all files",
                                                        "*.*")))

        count=1
        
        for jbutton in self.button_frame.winfo_children():
            jbutton.destroy()
        
        if filename.endswith('.xlsx'):
            self.label2.configure(text=f"File Opened: {(filename.split('/')[-1])}")
            df_words = pd.read_excel(filename,header=None, sheet_name = 1)
            count = 1
            df_leng = df_words.shape[0]
            word_row = 0

            if df_words.shape[1] < 3:
                df_words[2] = 0
            if df_words.shape[1] == 3:
                if 0 in set(df_words[2]):
                    word_row = df_words[df_words[2][0:] == 0].index.tolist()
                else:
                    word_row = "Done"

            wb = load_workbook(filename, read_only=False, keep_links=False)
            decks = wb.sheetnames
            ws = wb[decks[1]]
            df_leng = df_words.shape[0]
            

            self.segm_button = ctk.CTkSegmentedButton(self.button_frame, values=decks, command=self.playing_deck)
            self.segm_button.pack(padx=15, pady=40, expand = True)
            self.button_frame.pack()
        else:
            self.label2.configure(text="Invalid file format. Please choose an Excel file.")

        self.exp_btn.configure(text="Chosse other File")
        self.label3 = ctk.CTkLabel(self, text=f'', font=custom_font)
        self.action_frame = ctk.CTkFrame(self)
        wb.save(filename)
        wb.close()

    def playing_deck(self,value):
        global df_words, word_row
        df_words = pd.read_excel(filename,header=None, sheet_name = value)
        if df_words.shape[1] < 3:
            df_words[2] = 0
        if df_words.shape[1] == 3:
            if 0 in set(df_words[2]):
                word_row = df_words[df_words[2][0:] == 0].index.tolist()
            else:
                word_row = "Done"

        
        self.label3.configure(text=f'This Deck you have {len(df_words.index)} card')
        self.label3.pack()

        for abutton in self.action_frame.winfo_children():
            abutton.destroy()

        actions=["Play Flashcards", "Match pairs", "Add cards"]
        wins = [Win2, Win3, Win4]
        for index, (action, win) in enumerate(zip(actions, wins)):
            self.butnew(action, index+1, win)
      

    def butnew(self, text, number, _class):
        new_button = ctk.CTkButton(self.action_frame, text=text, command=lambda: self.new_window(number, _class,text))
        new_button.pack(side= "left", padx=5, pady=5, expand = True)
        self.action_frame.pack()

    def new_window(win1, number, _class,game):
        win1.new = ctk.CTkToplevel(win1.master)
        win1.new.grab_set()
        win1.new.geometry("700x500")
        win1.new.title(game)
        _class(win1.new, number)


class Win2:
    def __init__(win2, master, number):
        global word_row, df_leng, ctn
        ctn = 0
        win2.master = master
        win2.master.geometry("700x400")
        win2.frame = ctk.CTkFrame(win2.master)
        win2.frame.pack()
        win2.flashcard = ctk.CTkCanvas(win2.frame,width=500, height=200, bg=BACKGROUND_COLOR, highlightthickness=4)
        if (len(list(df_words[df_words[2] == 0].index))) != 0:
            win2.cv_button = win2.flashcard.create_text(250,80,text=df_words[0].iloc[word_row[0]], fill="black", font=("Ariel", 22, "bold"), width=280)
        else:
            win2.cv_button = win2.flashcard.create_text(220,80,text="Congratulations! You finished the deck!", fill="red", font=("Ariel", 22, "bold"), width=280)
            win2.button_destr(unknown_button, known_button)
            win2.quit = ctk.CTkButton(win2.frame, text = f"Quit this window", command = win2.close_window, fg_color="#7BA57B")

        win2.flashcard.tag_bind(win2.cv_button, "<Button-1>", win2.shift)# win2.shift(event))
        win2.flashcard.pack(side="top", pady= 100, ipadx=100)

        win2.unknown_button = ctk.CTkButton(win2.frame, text="Send back to deck",image=unknown_image, command= lambda: win2.lernt(0))#, compound= "bottom")#, command=word_unknown)
        win2.unknown_button.pack(side='left', anchor='se', padx=10, pady=10)

        win2.known_button = ctk.CTkButton(win2.frame,text="Learned",image=known_image, command=lambda: win2.lernt("learned"))# command=  #
        win2.known_button.pack(side='right', anchor='se', padx=10, pady=10)

    def button_destr(win2,*args):
        for x in args:
            x.destroy()


    def lernt(win2,state):
        global word_row, ctn
        df_words.loc[word_row[ctn],2] = state
        if ctn + 1 < len(word_row):
            ws.cell(row=word_row[ctn + 1], column=3).value = state
        if ctn + 1 == len(word_row):
            win2.flashcard.itemconfig(win2.cv_button, text="Congratulations! You finished the deck!", fill= "red")
            win2.button_destr(win2.unknown_button, win2.known_button)
            win2.quit2 = ctk.CTkButton(win2.frame, text = f"Quit this window", command = win2.close_window, fg_color="#7BA57B")
            win2.quit2.pack(side="bottom",ipady=10, padx=25, pady=25)
            word_row = df_words[df_words[2] == 0].index.tolist()
            ctn = 0
            if len(word_row) >= 1 :
                word_row = df_words[df_words[2] == 0].index.tolist()
                win2.flashcard.itemconfig(win2.cv_button, text=df_words[0].iloc[word_row[ctn]],fill="black")
                win2.unknown_button = ctk.CTkButton(win2.frame, text="Send back to deck",image=unknown_image, command= lambda: win2.lernt(0))#, compound= "bottom")#, command=word_unknown)
                win2.unknown_button.pack(side='left', anchor='se', padx=10, pady=10)

                win2.known_button = ctk.CTkButton(win2.frame,text="Learned",image=known_image, command=lambda: win2.lernt("learned"))# command=  #
                win2.known_button.pack(side='right', anchor='se', padx=10, pady=10)
                win2.button_destr(win2.quit2)
                word_row = df_words[df_words[2] == 0].index.tolist()
        else:
            ctn = ctn+1
            win2.flashcard.itemconfig(win2.cv_button, text=df_words[0].iloc[word_row[ctn]])
            
    def shift(win2, event):
        global word_row, count, ctn
        count = count + 1
        if count % 2 == 0:
            win2.flashcard.itemconfigure(win2.cv_button, text=df_words[1].iloc[word_row[ctn]])
        else:
            win2.flashcard.itemconfig(win2.cv_button, text=df_words[0].iloc[word_row[ctn]])

    def close_window(win2):
        win2.master.destroy()

class Win3:
    def __init__(win3, master, number):
        global rand_list
        win3.master = master
        win3.master.geometry("800x800")
        win3.frame = ctk.CTkFrame(win3.master)
        win3.frame.configure(fg_color="#66cdaa")
        win3.frame.pack(padx=20, pady=20)
        
        rand_list = []
        rs = random.sample(range(df_words.shape[0]), df_words.shape[0])

        for pair in range(df_words.shape[0]):
            rand_list.append([df_words.iloc[rs[pair]][0],df_words.iloc[rs[pair]][1]])

        win3.quit = ctk.CTkButton(win3.frame, text = f"Quit this window", command = win3.close_window, fg_color="#7BA57B")
        win3.label_match = ctk.CTkLabel(win3.frame, text=f"Match!",font=custom_font, wraplength=400)
        win3.label_select = ctk.CTkLabel(win3.frame, text=f"Select how many words you would like to play in one round",font=custom_font, wraplength=400, fg_color="#66CDAA")
        win3.input_label = ctk.CTkLabel(win3.frame, text="", font=custom_font, wraplength=400, fg_color="#66CDAA")
        win3.label_select.pack(padx=30, pady=10)
        win3.input_label.pack(padx=10, pady=10)

        win3.pair_butn_frame_l = ctk.CTkFrame(win3.frame)
        win3.pair_butn_frame_r = ctk.CTkFrame(win3.frame)

        win3.my_entry = ctk.CTkEntry(win3.frame, 
            placeholder_text="Enter a number",
            height=50,
            width=200,
            font=("Helvetica", 18),
            corner_radius=50,
            text_color="green", # outer, inner
            state="normal",
        )
        win3.my_entry.pack(pady=10)

        win3.submi_button = ctk.CTkButton(win3.frame, text="Submit", command= lambda: win3.submit())
        win3.submi_button.pack(pady=5)

        win3.clear_button = ctk.CTkButton(win3.frame, text="Clear", command= lambda: win3.clear())
        win3.clear_button.pack(pady=10)

        win3.quit.pack(side="bottom",ipady=10, padx=25, pady=25)

    def new_round(win3,inputlist):
        global inps_list, match
        match =0
        if 2*inputlist[1] > len(rand_list):
                inps_list = [inputlist[1],len(rand_list)]
        else:
                inps_list =list(map(lambda x: x + inputlist[1], inputlist))
        win3.pair_button(inps_list)
        win3.label_match.pack_forget()
        win3.new_pack_button.pack_forget()
        for disappear3 in ids_k_list: disappear3.destroy()
        ids_k_list.clear()

    def which_button(win3,button_pressed,id,c,val):
        global  clicked_list, ids_k_list, colm_val, match
        id[c].configure(fg_color=("blue"))
        if win3.label_match.winfo_ismapped() == True:
            win3.label_match.pack_forget()
            for disappear in ids_k_list: disappear.destroy()
            ids_k_list=[id[c]]
        else:
            ids_k_list.append(id[c])
        if clicked_list:
            if val in colm_val:
                for uncolor in ids_k_list[:-1]: uncolor.configure(fg_color=("#2F4F4F"))
                ids_k_list=[id[c]]
                colm_val=[val]
                clicked_list.append(button_pressed)
            else:
                colm_val.append(val)
                if button_pressed == clicked_list[-1:][0]:
                    win3.label_match.pack(padx=25, pady=25)
                    match=match+1
                else:
                    clicked_list.append(button_pressed)
        else:
            clicked_list.append(button_pressed)
            colm_val.append(val)
        if inps_list[1]-inps_list[0] == match:
            #for disappear2 in ids_k_list: disappear2.pack_forget()#.destroy()
            win3.pair_butn_frame_l.pack_forget()
            win3.pair_butn_frame_r.pack_forget()
            win3.new_pack_button = ctk.CTkButton(win3.frame, text="Next round", command=lambda: win3.new_round(inps_list))
            win3.new_pack_button.pack(padx=10, pady=10)
        if inps_list[1] == len(rand_list) and inps_list[1]-inps_list[0] == match:
            win3.new_pack_button.destroy()
            win3.quit = ctk.CTkButton(win3.frame, text = f"Quit this window")#, command = win4.close_window)
            win3.quit.pack()



    def pair_button(win3,nu):
        global rand_list
        # user val az legyen, ahány kártyát kér a user = nu[1]- nu[0] - mennyi köztük a különbség - kivéve, ha már nincs annyi
        diff = nu[1]-nu[0]
        pair_rand2 =random.sample(range(diff),diff)
        win3.input_label.configure(text=f'Match the pairs of {nu[1]} cards', font=("Helvetica", 20))
        # ha egyszer túl megy az összes limiten, akkor már nem tud új roundot kérni
        pair_butt_a = [[None]*1 for _ in range(nu[1])]
        pair_butt_b = [[None]*1 for _ in range(nu[1])]
        start, end = nu[0], nu[1]
        for rag,show_fisrt,show_sec in zip(range(start, end),rand_list[start:end],pair_rand2):
            pair_butt_a[rag] = ctk.CTkButton(win3.pair_butn_frame_l, text=show_fisrt[0],command=lambda c=rag, m= show_fisrt: win3.which_button(m,pair_butt_a,c,"a"))
            pair_butt_b[rag] = ctk.CTkButton(win3.pair_butn_frame_r, text=rand_list[start:end][show_sec][1],command=lambda c=rag, m= rand_list[start:end][show_sec]: win3.which_button(m,pair_butt_b,c,"b"))
            pair_butt_a[rag].pack(padx=15, pady=5, expand = True)
            pair_butt_b[rag].pack(padx=15, pady=5, expand = True)
            win3.pair_butn_frame_l.pack(side="left", padx=25, pady=25)
            win3.pair_butn_frame_r.pack(side="right", padx=25, pady=25)


    def submit(win3,event=None):
        global int_value, inps_list
        win3.label_select.pack_forget()
        user_input = win3.my_entry.get()
        try:
            int_value = int(user_input)
            win3.input_label.configure(text=f"The input is an integer: {int_value}")
            inps_list = [0,int_value]
            win3.pair_button(inps_list)
            win3.clear_button.destroy()
            win3.submi_button.destroy()
            win3.my_entry.pack_forget()
        except ValueError:
            try:
                float_value = float(user_input)
                win3.input_label.configure(text=f"The input is a float: {float_value}\n"
             f'Ensure a valid number')
            except ValueError:
                win3.input_label.configure(text=f"The input is a string: {user_input}\n"
             f'Ensure a valid number')
                
    def clear(win3,event=None):
        win3.my_entry.configure(state="normal")
        win3.my_entry.delete(0, ctk.END)
        win3.input_label.configure(text=f'Add an input')

    def close_window(win3):
        win3.master.destroy()


class Win4:
    def __init__(win4, master, number):
        global add_cards
        win4.master = master
        win4.master.geometry("900x200")
        

        win4.scrollable_frame = ctk.CTkScrollableFrame(win4.master, width=780, height=300)
        win4.scrollable_frame.pack(padx=10, pady=10, fill="both", expand=True)

        win4.bot_frame = ctk.CTkFrame(win4.master)
        win4.bot_frame.pack(side = ctk.BOTTOM, padx=10, pady=10, fill="both", expand=True)

        win4.quit = ctk.CTkButton(win4.scrollable_frame, text = f"Quit this window", command = win4.close_window, fg_color="#7BA57B")
        win4.quit.pack(side=ctk.BOTTOM, padx=10, pady=10, fill="both")

        win4.add = ctk.CTkButton(win4.scrollable_frame, text = f"Add new word", command = win4.add_new_row, fg_color="#7BA57B")
        win4.add.pack(side=ctk.BOTTOM, padx=10, pady=10, fill="both")

        win4.input_field1 = ctk.CTkEntry(win4.bot_frame, placeholder_text="Type something...")
        win4.input_field1.pack(side="left")

        win4.input_field2 = ctk.CTkEntry(win4.bot_frame, placeholder_text="Type something...")
        win4.input_field2.pack(side="right")

        win4.f1frame = ctk.CTkFrame(win4.scrollable_frame)
        win4.f1frame.pack(side="left", padx=10)

        win4.f2frame = ctk.CTkFrame(win4.scrollable_frame)
        win4.f2frame.pack(side="right", padx=10)
    
        win4.master.after(0, win4.add_words_to_display(df_words))

    def add_new_row(win4):
        input_field = ctk.CTkEntry(win4.scrollable_frame, placeholder_text="Type something...")
        input_field.pack(side=ctk.BOTTOM, padx=10, pady=10)

    def add_words_to_display(win4,my_df):
        global add_cards
        
        add_cards = []
        add_cards2 = []

        for i in range((my_df.shape[0])):
            win4.new_pack_button = ctk.CTkButton(win4.f2frame, text="X", command= lambda j=i: win4.delete_row(j), fg_color="#D05B5B")
            win4.new_pack_button.pack(pady = 5, expand=True)
            add_cards2.append(win4.new_pack_button)

        add_cards.append(add_cards2)

        sides = ["left", "right"]
        
        for row_index in range(2):
            add_cards3 = []
            le_col_frame = ctk.CTkFrame(win4.f1frame)
            le_col_frame.pack(side = sides[row_index], padx=10)
            add_cards2.append(add_cards3)
            for col_index in range(my_df.shape[0]):
                wt = my_df[row_index][col_index]
                win4.word_label = ctk.CTkLabel(le_col_frame, text= wt, width=10)
                win4.word_label.pack(pady = 5, expand=True)
                add_cards3.append(win4.word_label)
            add_cards.append(add_cards3)
        
    
    def close_window(win4):
        win4.master.destroy()

    def display_label(win4):
        if win4.label is None:
            win4.label = ctk.CTkLabel(win4.frame, text="Button was clicked!")
            win4.label.pack()
    
    def delete_row(win4,num):
        global add_cards
        for k in range(3):
            add_cards[k][num].destroy()

            
def main():
    # Create the main GUI window
    ctk.set_default_color_theme('./self_theme2.json')
    root = ctk.CTk()
    root.title('Flashcards App')
    root.geometry('500x400')
    
    window = Main(root)
    window.pack()
    root.mainloop()


if __name__ == '__main__':
    
    main()